<a href="https://colab.research.google.com/github/satani99/practical_deep_learning_for_coders/blob/main/fast_ai_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [2]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user', 'movie', 'rating', 'timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
last_skywalker = np.array([0.98, 0.9, -0.9])

In [4]:
user1 = np.array([0.9, 0.8, -0.6])

In [6]:
(user1@last_skywalker.T)

2.1420000000000003

In [7]:
casablanca = np.array([-0.99, -0.3, 0.8])

In [8]:
(user1@casablanca.T)

-1.611

In [9]:
movies = pd.read_csv(path/'u.item', delimiter='|', encoding='latin-1',
                     usecols=(0, 1), names=('movie', 'title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [10]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [11]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,30,Twelve Monkeys (1995),4
1,58,"Usual Suspects, The (1995)",5
2,225,North by Northwest (1959),5
3,493,"Empire Strikes Back, The (1980)",5
4,621,Home Alone (1990),2
5,89,"Godfather: Part II, The (1974)",5
6,886,"Phantom, The (1996)",1
7,250,Fargo (1996),5
8,381,Dances with Wolves (1990),4
9,276,Toy Story (1995),5


In [13]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors = 5

user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

In [14]:
one_hot_3 = one_hot(3, n_users).float()
user_factors.t() @ one_hot_3

tensor([-1.3361, -1.0442, -1.1789,  0.4317,  0.4034])

In [15]:
user_factors[3]

tensor([-1.3361, -1.0442, -1.1789,  0.4317,  0.4034])

In [25]:
class DotProduct(Module):
  def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
    self.user_factors = Embedding(n_users, n_factors)
    self.user_bias = Embedding(n_users, 1)
    self.movie_factors = Embedding(n_movies, n_factors)
    self.movie_bias = Embedding(n_movies, 1)
    self.y_range = y_range

  def forward(self, x):
    users = self.user_factors(x[:, 0])
    movies = self.movie_factors(x[:, 1])
    res = (users * movies).sum(dim=1, keepdim=True)
    res += self.user_bias(x[:,0]) + self.movie_bias(x[:,1])
    return sigmoid_range(res, *self.y_range)

In [26]:
x, y = dls.one_batch()
x.shape

torch.Size([64, 2])

In [27]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [31]:
learn.fit_one_cycle(5, 5e-3, wd=0.2)

epoch,train_loss,valid_loss,time
0,0.454869,0.880911,00:10
1,0.683845,0.866245,00:09
2,0.645227,0.848623,00:09
3,0.582065,0.836171,00:09
4,0.456158,0.835362,00:09


In [34]:
class T(Module):
  def __init__(self): self.a = nn.Linear(1, 3, bias=False)

t = T()
L(t.parameters())

(#1) [Parameter containing:
tensor([[-0.7287],
        [ 0.4026],
        [-0.1992]], requires_grad=True)]

In [35]:
type(t.a.weight)

torch.nn.parameter.Parameter

In [36]:
def create_params(size):
  return nn.Parameter(torch.zeros(*size).normal_(0, 0.01))

In [39]:
class DotProductBias(Module):
  def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
    self.user_factors = create_params([n_users, n_factors])
    self.user_bias = create_params([n_users])
    self.movie_factors = create_params([n_movies, n_factors])
    self.movie_bias = create_params([n_movies])
    self.y_range = y_range

  def forward(self, x):
    users = self.user_factors[x[:,0]]
    movies = self.movie_factors[x[:,1]]
    res = (users*movies).sum(dim=1)
    res += self.user_bias[x[:,0]] + self.movie_bias[x[:,1]]
    return sigmoid_range(res, *self.y_range)

In [40]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.916878,0.941890,00:11
1,0.870444,0.868704,00:09
2,0.753615,0.835469,00:09
3,0.575761,0.825995,00:09
4,0.479533,0.827561,00:09


In [41]:
movie_bias = learn.model.movie_bias.squeeze()
idxs = movie_bias.argsort()[:5]
[dls.classes['title'][i] for i in idxs]

['Children of the Corn: The Gathering (1996)',
 'Island of Dr. Moreau, The (1996)',
 'Lawnmower Man 2: Beyond Cyberspace (1996)',
 'Amityville II: The Possession (1982)',
 'Big Bully (1996)']

In [44]:
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]

['Titanic (1997)',
 'L.A. Confidential (1997)',
 "Schindler's List (1993)",
 'Star Wars (1977)',
 'Shawshank Redemption, The (1994)']

In [45]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.960325,0.941072,00:15
1,0.867261,0.879949,00:09
2,0.713608,0.828290,00:09
3,0.609439,0.822171,00:08
4,0.462781,0.822017,00:08


In [46]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(944, 50)
  (i_weight): Embedding(1665, 50)
  (u_bias): Embedding(944, 1)
  (i_bias): Embedding(1665, 1)
)

In [48]:
movie_bias = learn.model.i_bias.weight.squeeze()
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]

['Titanic (1997)',
 'L.A. Confidential (1997)',
 'Star Wars (1977)',
 "Schindler's List (1993)",
 'Good Will Hunting (1997)']